# Client Project: The Lab @ DC

## Project Title: {here}

### Authors: {names}
- Cohorts of the Data Science Immersive, General Assembly @ Washington DC campus

In this notebook, we have Exploratory Data Analysis on the datasets. **This is notebook 2 of 3.**

### Import Libraries

In [1]:
# import basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Read CSVs

In [2]:
csr_train   = pd.read_csv('./assets/csr/csr_train.csv', low_memory=False)
shots_train = pd.read_csv('./assets/mpd/shots_train.csv', low_memory=False)
shots_test  = pd.read_csv('./assets/mpd/shots_test.csv', low_memory=False)

### Basic EDAs and Data Cleaning

In [3]:
csr_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231233 entries, 0 to 1231232
Data columns (total 31 columns):
Unnamed: 0                    1231233 non-null int64
X                             1231233 non-null float64
Y                             1231233 non-null float64
OBJECTID                      1231233 non-null int64
SERVICECODE                   1231233 non-null object
SERVICECODEDESCRIPTION        1231233 non-null object
SERVICETYPECODEDESCRIPTION    1230379 non-null object
ORGANIZATIONACRONYM           1231232 non-null object
SERVICECALLCOUNT              1231233 non-null int64
ADDDATE                       1231233 non-null object
RESOLUTIONDATE                1145187 non-null object
SERVICEDUEDATE                1218530 non-null object
SERVICEORDERDATE              1231233 non-null object
INSPECTIONFLAG                1231233 non-null object
INSPECTIONDATE                434130 non-null object
INSPECTORNAME                 40361 non-null object
SERVICEORDERSTATUS         

In [4]:
csr_train.isnull().sum().sort_values(ascending=False)

INSPECTORNAME                 1190872
INSPECTIONDATE                 797103
DETAILS                        444580
MARADDRESSREPOSITORYID         189162
STATUS_CODE                    151801
RESOLUTIONDATE                  86046
CITY                            50324
STATE                           50324
STREETADDRESS                   49730
SERVICEDUEDATE                  12703
WARD                             6221
PRIORITY                         2677
SERVICETYPECODEDESCRIPTION        854
SERVICEORDERSTATUS                853
ZIPCODE                            16
ORGANIZATIONACRONYM                 1
SERVICECALLCOUNT                    0
SERVICECODE                         0
OBJECTID                            0
Y                                   0
X                                   0
SERVICECODEDESCRIPTION              0
INSPECTIONFLAG                      0
ADDDATE                             0
SERVICEORDERDATE                    0
SERVICEREQUESTID                    0
XCOORD      

##### Teams, two datasets have different digits in the Lat's and Long's. Will it be matters? 

In [5]:
csr_train[['LATITUDE', 'LONGITUDE']].head()

,LATITUDE,LONGITUDE
0,38.897950,-76.972732
1,38.922857,-76.991905
2,38.874742,-76.970889
3,38.942811,-77.022676
4,38.898952,-77.048838


In [6]:
shots_train[['Latitude', 'Longitude']].head()

,Latitude,Longitude
0,38.917,-77.012
1,38.917,-77.002
2,38.917,-76.987
3,38.823,-77.000
4,38.893,-76.993
